# Install dependencies

* IMPORTANT: you only need to run this one time, when you first spin up your docker notebook server
* IMPORTANT: you need to restart the kernel after you run this
* TODO: these deps should be eventually put into a dockerfile

* INFO: read more about [what the hell is pip](https://pip.pypa.io/en/stable/) and [why installing dependencies via pip in a notebook is a stupid idea, but OK since we are prototyping here](https://towardsdev.com/pip-install-from-jupyter-notebook-485c218b50b)

In [ ]:
!pip install pandas
!pip install datasets

# Import libraries

also setup the data directory

In [1]:
import os
import re
import glob
import shutil
import string
import pathlib

# set the data directory
data_dir = os.path.abspath(os.path.join(os.getcwd(),'..','data'))

#give matplotlib a folder to save its configs
os.environ['MPLCONFIGDIR'] = os.path.join(data_dir,'plt_configs')
import matplotlib.pyplot as plt

#give huggingface a folder to save its stuff in too
#you only need this if you are using a huggingface dataset
os.environ['HF_HOME'] = os.path.join(data_dir,'hf_cache')
import datasets

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.layers import TextVectorization

# Download the data

* [huggingface datasets quickstart](https://huggingface.co/docs/datasets/quickstart)
* [joelito/brazilian_court_decisions](https://huggingface.co/datasets/joelito/brazilian_court_decisions)

In [2]:
from datasets import load_dataset

dataset = load_dataset("joelito/brazilian_court_decisions")

Using custom data configuration joelito--brazilian_court_decisions-cc57c1c8c69e3b04
Found cached dataset json (/tf/data/hf_cache/datasets/joelito___json/joelito--brazilian_court_decisions-cc57c1c8c69e3b04/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

# Inspect the data 

* [Know your dataset](https://huggingface.co/docs/datasets/access)

In [10]:
train_df = dataset['train'].to_pandas()
train_df.head()

,process_number,orgao_julgador,publish_date,judge_relator,ementa_text,decision_description,judgment_text,judgment_label,unanimity_text,unanimity_label
0,0800304-08.2018.8.02.0000,Tribunal Pleno,12/03/2019,Des. João Luiz Azevedo Lessa,DIREITO PENAL E PROCESSUAL PENAL. REVISÃO CRIM...,DIREITO PENAL E PROCESSUAL PENAL. REVISÃO CRIM...,REVISÃO CRIMINAL JULGADA PARCIALMENTE PROCEDENTE,partial,DECISÃO UNÂNIME,unanimity
1,0700071-82.2015.8.02.0040,3ª Câmara Cível,09/02/2019,Des. Celyrio Adamastor Tenório Accioly,APELAÇÃO CÍVEL. MANDADO DE SEGURANÇA. SERVIDOR...,APELAÇÃO CÍVEL. MANDADO DE SEGURANÇA. SERVIDOR...,RECURSO CONHECIDO E NÃO PROVIDO,no,-2,not_determined
2,0801729-70.2018.8.02.0000,2ª Câmara Cível,25/02/2019,Des. Pedro Augusto Mendonça de Araújo,PROCESSUAL CIVIL. EMBARGOS DE DECLARAÇÃO EM AG...,PROCESSUAL CIVIL. EMBARGOS DE DECLARAÇÃO EM AG...,RECURSO CONHECIDO E REJEITADO,no,DECISÃO UNÂNIME,unanimity
3,0804894-62.2017.8.02.0000,2ª Câmara Cível,19/03/2019,Des. Klever Rêgo Loureiro,AGRAVO DE INSTRUMENTO. AUXÍLIO DOENÇA. SUSPENS...,AGRAVO DE INSTRUMENTO. AUXÍLIO DOENÇA. SUSPENS...,RECURSO CONHECIDO E PROVIDO,yes,DECISÃO UNÂNIME,unanimity
4,0702761-41.2014.8.02.0001,1ª Câmara Cível,13/02/2019,Des. Fábio José Bittencourt Araújo,DIREITO DO CONSUMIDOR. APELAÇÃO INTERPOSTA EM ...,DIREITO DO CONSUMIDOR. APELAÇÃO INTERPOSTA EM ...,APELO CONHECIDO E PROVIDO EM PARTE,partial,UNANIMIDADE,unanimity


In [12]:
test_df = dataset['test'].to_pandas()
test_df.head()

,process_number,orgao_julgador,publish_date,judge_relator,ementa_text,decision_description,judgment_text,judgment_label,unanimity_text,unanimity_label
0,0727978-86.2014.8.02.0001,1ª Câmara Cível,06/02/2019,Des. Fábio José Bittencourt Araújo,DIREITO ADMINISTRATIVO. APELAÇÃO CÍVEL. SENTEN...,DIREITO ADMINISTRATIVO. APELAÇÃO CÍVEL. SENTEN...,RECURSO CONHECIDO E PARCIALMENTE PROVIDO,partial,UNANIMIDADE,unanimity
1,0701144-98.2016.8.02.0058,3ª Câmara Cível,17/12/2018,Des. Alcides Gusmão da Silva,APELAÇÕES CÍVEIS. AÇÃO ORDINÁRIA COM PEDIDO DE...,APELAÇÕES CÍVEIS. AÇÃO ORDINÁRIA COM PEDIDO DE...,RECURSO CONHECIDO E PROVIDO,yes,-2,not_determined
2,0706293-81.2018.8.02.0001,Câmara Criminal,07/03/2019,Des. Sebastião Costa Filho,PENAL. PROCESSO PENAL. APELAÇÃO CRIMINAL. RECE...,PENAL. PROCESSO PENAL. APELAÇÃO CRIMINAL. RECE...,III - Apelação conhecida e improvida,no,-2,not_determined
3,0806267-94.2018.8.02.0000,2ª Câmara Cível,20/03/2019,Desa. Elisabeth Carvalho Nascimento,AGRAVO DE INSTRUMENTO. AÇÃO DE BUSCA E APREENS...,AGRAVO DE INSTRUMENTO. AÇÃO DE BUSCA E APREENS...,RECURSO CONHECIDO E NÃO PROVIDO,no,-2,not_determined
4,0700303-14.2017.8.02.0044,2ª Câmara Cível,29/01/2019,Desa. Elisabeth Carvalho Nascimento,APELAÇÃO CÍVEL. MANDADO DE SEGURANÇA. REINTEGR...,APELAÇÃO CÍVEL. MANDADO DE SEGURANÇA. REINTEGR...,RECURSO CONHECIDO E NÃO PROVIDO,no,-2,not_determined


# Setup a pandas dataframe, tokenize etc
* [Load a pandas DataFrame](https://www.tensorflow.org/tutorials/load_data/pandas_dataframe)
* [Load text](https://www.tensorflow.org/tutorials/load_data/text)

In [ ]:
# TODO, add code here :)

# Load data and train model
* [tf text in 5 mins](https://codesearchonline.com/natural-language-processing-with-tensorflow-cheat-sheet/)

In [ ]:
# TODO add code here

# Save model and test prediction

In [ ]:
# TODO add code here